In [73]:
import pandas as pd
import numpy as np
import os
import requests


In [74]:
######################## CRASHES ###########################

COLUMNS_TO_DROP_CRASHES = [
    "CRASH_DATE_EST_I",
    "LANE_CNT",
    "INTERSECTION_RELATED_I",
    "NOT_RIGHT_OF_WAY_I",
    "HIT_AND_RUN_I",
    "PHOTOS_TAKEN_I",
    "STATEMENTS_TAKEN_I",
    "DOORING_I",
    "WORK_ZONE_I",
    "WORK_ZONE_TYPE",
    "WORKERS_PRESENT_I",
    "CRASH_HOUR",
    "CRASH_DAY_OF_WEEK",
    "CRASH_MONTH",
    "LOCATION",
]

# tu nulle UNKNOWN
COLUMNS_TO_STRING_CRASHES = [
    "TRAFFIC_CONTROL_DEVICE",
    "DEVICE_CONDITION",
    "WEATHER_CONDITION",
    "LIGHTING_CONDITION",
    "FIRST_CRASH_TYPE",
    "TRAFFICWAY_TYPE",
    "ALIGNMENT",
    "ROADWAY_SURFACE_COND",
    "ROAD_DEFECT",
    "REPORT_TYPE",
    "CRASH_TYPE",
    "DAMAGE",
    "PRIM_CONTRIBUTORY_CAUSE",
    "SEC_CONTRIBUTORY_CAUSE",
    "STREET_DIRECTION",
    "STREET_NAME",
    "MOST_SEVERE_INJURY",
]

COLUMNS_TO_INT_CRASHES = [
    "POSTED_SPEED_LIMIT",
    "STREET_NO",
    "BEAT_OF_OCCURRENCE",
    "NUM_UNITS",
    "INJURIES_TOTAL",
    "INJURIES_FATAL",
    "INJURIES_INCAPACITATING",
    "INJURIES_NON_INCAPACITATING",
    "INJURIES_REPORTED_NOT_EVIDENT",
    "INJURIES_NO_INDICATION",
    "INJURIES_UNKNOWN",
]

COLUMNS_TO_FLOAT_CRASHES = ["LATITUDE", "LONGITUDE"]

COLUMNS_TO_DATE = [
    "CRASH_DATE",
    "DATE_POLICE_NOTIFIED",
]

COLUMNS_TO_DIM_CRASH_INFO = [
    "CRASH_RECORD_ID",
    "TRAFFIC_CONTROL_DEVICE",
    "DEVICE_CONDITION",
    "FIRST_CRASH_TYPE",
    "TRAFFICWAY_TYPE",
    "ALIGNMENT",
    "ROADWAY_SURFACE_COND",
    "ROAD_DEFECT",
    "REPORT_TYPE",
    "CRASH_TYPE",
    "DAMAGE",
    "PRIM_CONTRIBUTORY_CAUSE",
    "SEC_CONTRIBUTORY_CAUSE",
    "MOST_SEVERE_INJURY",
]

COLUMNS_TO_FACT_CRASH = [
    "CRASH_RECORD_ID",
    "NUM_UNITS",
    "INJURIES_TOTAL",
    "INJURIES_FATAL",
    "INJURIES_INCAPACITATING",
    "INJURIES_NON_INCAPACITATING",
    "INJURIES_REPORTED_NOT_EVIDENT",
    "INJURIES_NO_INDICATION",
    "INJURIES_UNKNOWN",
]

COLUMNS_TO_DIM_LOCATION = [
    "CRASH_RECORD_ID",
    "STREET_DIRECTION",
    "STREET_NAME",
    "STREET_NO",
    "LATITUDE",
    "LONGITUDE"
]

###################################### PEOPLE ########################################

COLUMNS_TO_DROP_PEOPLE = [ 
    "SEAT_NO",           
    "CITY",              
    "STATE",               
    "ZIPCODE",             
    "DRIVERS_LICENSE_STATE",
    "DRIVERS_LICENSE_CLASS",
    "HOSPITAL",            
    "EMS_AGENCY",
    "EMS_RUN_NO",
    "PEDPEDAL_ACTION",
    "PEDPEDAL_VISIBILITY",
    "PEDPEDAL_LOCATION",
    "CELL_PHONE_USE",
    ]

COLUMNS_TO_STRING_PEOPLE = [
    "PERSON_ID",
    "PERSON_TYPE",
    "CRASH_RECORD_ID",
    "SEX",
    "SAFETY_EQUIPMENT",
    "AIRBAG_DEPLOYED",
    "EJECTION",
    "INJURY_CLASSIFICATION",
    "DRIVER_ACTION",
    "DRIVER_VISION",
    "PHYSICAL_CONDITION",
    "BAC_RESULT" 
]

COLUMNS_TO_INT_PEOPLE = [
    # "VEHICLE_ID",
    "AGE",
]

COLUMNS_TO_FLOAT_PEOPLE = [
    "BAC_RESULT VALUE"
]

############## VEHICLES #################

COLUMNS_TO_DROP_VEHICLES = [
    "CRASH_DATE",
    "UNIT_NO",
    #   "VEHICLE_ID",  ?
    "CMRC_VEH_I",
    "LIC_PLATE_STATE",
    "TOWED_I",
    "FIRE_I",
    "EXCEED_SPEED_LIMIT_I",
    "TOWED_BY",
    "TOWED_TO",
    "AREA_00_I",
    "AREA_01_I",
    "AREA_02_I",
    "AREA_03_I",
    "AREA_04_I",
    "AREA_05_I",
    "AREA_06_I",
    "AREA_07_I",
    "AREA_08_I",
    "AREA_09_I",
    "AREA_10_I",
    "AREA_11_I",
    "AREA_12_I",
    "AREA_99_I",
    "CMV_ID",
    "USDOT_NO",
    "CCMC_NO",
    "ILCC_NO",
    "COMMERCIAL_SRC",
    "GVWR",
    "CARRIER_NAME",
    "CARRIER_STATE",
    "CARRIER_CITY",
    "HAZMAT_PLACARDS_I",
    "HAZMAT_NAME",
    "UN_NO",
    "HAZMAT_PRESENT_I",
    "HAZMAT_REPORT_I",
    "HAZMAT_REPORT_NO",
    "MCS_REPORT_I",
    "MCS_REPORT_NO",
    "HAZMAT_VIO_CAUSE_CRASH_I",
    "MCS_VIO_CAUSE_CRASH_I",
    "IDOT_PERMIT_NO",
    "WIDE_LOAD_I",
    "TRAILER1_WIDTH",
    "TRAILER2_WIDTH",
    "TRAILER1_LENGTH",
    "TRAILER2_LENGTH",
    "TOTAL_VEHICLE_LENGTH",
    "AXLE_CNT",
    "VEHICLE_CONFIG",
    "CARGO_BODY_TYPE",
    "LOAD_TYPE",
    "HAZMAT_OUT_OF_SERVICE_I",
    "MCS_OUT_OF_SERVICE_I",
    "HAZMAT_CLASS",
]

COLUMNS_TO_STRING_VEHICLES = [
    "UNIT_TYPE",
    "MAKE",
    "MODEL",
    "VEHICLE_DEFECT",
    "VEHICLE_TYPE",
    "VEHICLE_USE",
    "TRAVEL_DIRECTION",
    "MANEUVER",
    "FIRST_CONTACT_POINT",
]

COLUMNS_TO_INT_VEHICLES = [
    "CRASH_UNIT_ID",
    # "VEHICLE_ID", # nie obsługujemy tego
    "NUM_PASSENGERS",
    "VEHICLE_YEAR",
    "OCCUPANT_CNT",  # DO FACT CRASH
]

############## WEATHER ##################

COLUMNS_TO_DROP_WEATHER = ["name", "severerisk", "icon", "stations"]

COLUMNS_NULL_UNKNOWN_WEATHER = ["conditions"]

COLUMNS_NULL_NONE_WEATHER = ["preciptype"]

COLUMNS_TO_FLOAT_WEATHER = [
    "temp",
    "feelslike",
    "dew",
    "humidity",
    "precip",
    "snow",
    "snowdepth",
    "windgust",
    "windspeed",
    "sealevelpressure",
    "cloudcover",
    "visibility",
    "solarradiation",
    "solarenergy",
]

COLUMNS_TO_INT_WEATHER = ["precipprob", "uvindex"]

In [71]:
import hashlib
import pandas as pd
from typing import List


def change_type(df: pd.DataFrame, cols: List[str], type: str) -> pd.DataFrame:
    """
    Changes the data type of specified columns in a DataFrame.

    Args:
        df (pd.DataFrame): Input DataFrame.
        cols (List[str]): List of column names to change the data type.
        type (str): Desired data type to convert the columns to.

    Returns:
        pd.DataFrame: DataFrame with specified columns converted to the desired data type.
    """
    df_copy = df.copy()
    for col in cols:
        if col in df_copy.columns:
            df_copy[col] = df_copy[col].astype(type)
    return df_copy


def fill_na(df: pd.DataFrame, cols: List[str], value) -> pd.DataFrame:
    """Fill missing values in specified columns with a given value.

    Args:
        df (pd.DataFrame): Input DataFrame.
        cols (List[str]): A list of column names where NA values should be filled.
        value (_type_): The value used to replace NA values.

    Returns:
        pd.DataFrame: A DataFrame with NA values in the specified columns filled.
    """
    df_copy = df.copy()
    for col in cols:
        if col in df_copy.columns:
            df_copy[col] = df_copy[col].fillna(value)
    return df_copy


def replace_value(
    df: pd.DataFrame, cols: List[str], value_in, value_out
) -> pd.DataFrame:
    """Replaces specified values with a given value_out.

    Args:
        df (pd.DataFrame): Input DataFrame.
        cols (List[str]): A list of column names where values should be changed.
        value_in (_type_): The value to be changed.
        value_out (_type_): The value used to replace value_in.

    Returns:
        pd.DataFrame: A DataFrame with changed value_in to value_out.
    """
    df_copy = df.copy()
    for col in cols:
        if col in df_copy.columns:
            df_copy[col] = df_copy[col].replace(value_in, value_out)
    return df_copy


def generate_surrogate_key(*args) -> int:
    """
    Generates a surrogate key based on the input arguments.

    Args:
        *args: Variable number of arguments to generate the key from.

    Returns:
        int: The generated surrogate key.
    """
    # example use:
    # dim_crash_info_df["crash_info_id"] = dim_crash_info_df.apply(lambda row: generate_surrogate_key(*[row[col] for col in crash_info_cols]), axis=1)

    key_string = "|".join(str(arg).lower().strip() for arg in args)
    hash_bytes = hashlib.sha256(key_string.encode("utf-8")).digest()

    bigint = int.from_bytes(hash_bytes[:4], byteorder="big", signed=False)

    return bigint


## Sprawdzanie po trans_formed

In [167]:
import os
os.getcwd()

'c:\\Users\\Tomek\\Desktop\\PW MINI IAD\\Semestr 6\\Hurtownie\\ChicagoCrashesETL'

In [ ]:
df_vehicles = pd.read_pickle('./airflow-postgres-docker/data/tmp/extracted/vehicles.pkl')
df_crashes = pd.read_pickle('./airflow-postgres-docker/data/tmp/extracted/crashes.pkl')

In [79]:
df = df_vehicles

df = df.drop(columns=COLUMNS_TO_DROP_VEHICLES)

# String handling
df = fill_na(df, COLUMNS_TO_STRING_VEHICLES, "UKNOWN")
df = replace_value(df, COLUMNS_TO_STRING_VEHICLES, "", "UNKNOWN")
df = change_type(df, COLUMNS_TO_STRING_VEHICLES, "string")

# Int handling
df = fill_na(df, COLUMNS_TO_INT_VEHICLES, -1)
df['OCCUPANT_CNT'] = df['OCCUPANT_CNT'] + 1
df = change_type(df, COLUMNS_TO_INT_VEHICLES, "Int64")

# Na szybko bo może zadziała (zrobić VEHICLE_ID jako CRASH_UNIT_ID bo to to samo)
df['VEHICLE_ID'] = df['CRASH_UNIT_ID']

print(sum(df.duplicated()))

df["VEHICLE_KEY"] = df.apply(
    lambda row: generate_surrogate_key(*[row[col] for col in df.columns.to_list()]),
    axis=1,
)

print(sum(df.duplicated(subset='VEHICLE_KEY')))

0
7


In [93]:
fact_crash = pd.read_pickle('./airflow-postgres-docker/data/tmp/transformed/fact_crash.pkl')
dim_vehicles = pd.read_pickle('./airflow-postgres-docker/data/tmp/transformed/dim_vehicle.pkl')
dim_crash_info = pd.read_pickle('./airflow-postgres-docker/data/tmp/transformed/dim_crash_info.pkl')
fact_weather = pd.read_pickle('./airflow-postgres-docker/data/tmp/transformed/fact_weather.pkl')
dim_people = pd.read_pickle('./airflow-postgres-docker/data/tmp/transformed/dim_people.pkl')
dim_date = pd.read_pickle('./airflow-postgres-docker/data/tmp/transformed/dim_date.pkl')
dim_location = pd.read_pickle('./airflow-postgres-docker/data/tmp/transformed/dim_location.pkl')

In [81]:
sum(fact_crash.duplicated('FACT_CRASH_KEY'))

0

In [95]:
all_vehicle_ids = dim_vehicles['VEHICLE_ID']

selected_from_crashes = fact_crash[fact_crash['VEHICLE_ID'].isin(all_vehicle_ids)]

In [96]:
print(len(selected_from_crashes))
print(len(fact_crash))


214323
214323


,CRASH_UNIT_ID,CRASH_RECORD_ID,UNIT_TYPE,NUM_PASSENGERS,VEHICLE_ID,MAKE,MODEL,VEHICLE_YEAR,VEHICLE_DEFECT,VEHICLE_TYPE,VEHICLE_USE,TRAVEL_DIRECTION,MANEUVER,OCCUPANT_CNT,FIRST_CONTACT_POINT,VEHICLE_KEY


In [82]:
sum(dim_location['LOCATION_KEY'].duplicated())

0

In [86]:
sum(dim_vehicles['VEHICLE_ID'].duplicated())

0